In [1]:
import pandas as pd
import re
import os

COLLECT DATA

In [4]:
def is_date(st):
    search_res = re.search('\w+day, \w+ \d{1,2}, \d{4}',st)
    if search_res:
        return True
    else:
        False

In [5]:
is_date("Sunday, August 15, 2021")

True

In [28]:
from datetime import datetime
# datetime.strptime('Jun 1 2005  1:33PM', '%b %d %Y %I:%M%p')

d = datetime.strptime('Sunday, August 15, 2021', '%A, %B %d, %Y')
d.isocalendar()

(2021, 32, 7)

In [7]:
df = pd.read_csv("FoodDiary_Aug_21_foods.csv")
df = df[['Date','Prot( g)','Fat( g)','Carbs( g)','Cals ( kcal)']]
df.columns = ['date','prot','fat','carbs','kcal']
df.head()

,date,prot,fat,carbs,kcal
0,"Sunday, August 1, 2021",80.21,65.58,257.34,1971.0
1,Breakfast,30.87,33.76,90.52,789.0
2,Вологодские Продукты Бифилайф Малина-Земляника,8.00,6.25,18.50,162.0
3,"2 1/2 x 100г, 250 g",NaN,NaN,NaN,NaN
4,Томилинская Форель Слабосоленая,13.80,9.00,0.00,136.0


In [118]:
from collections import OrderedDict, defaultdict

def get_month_dict(path):
    df_c = pd.read_csv(path)
    df_c = df_c[['Date','Prot( g)','Fat( g)','Carbs( g)','Cals ( kcal)']]
    df_c.columns = ['date','prot','fat','carbs','kcal']
    feature_columns = ['prot','fat','carbs','kcal']
    for _, el in df_c.iterrows():
        if is_date(el['date']):
            parsed_date = datetime.strptime(el['date'], '%A, %B %d, %Y')
            current_week, current_weekday = parsed_date.isocalendar()[1:]
            current_date_raw = el['date']
            
            DATE_STORAGE[current_week][current_weekday] = {'auto_sum':el[feature_columns].tolist(),'raw_date':current_date_raw}
        elif el['date'].strip() in ('Breakfast','Lunch','Dinner','Snacks/Other'):
            DATE_STORAGE[current_week][current_weekday][el['date'].strip()] = el[feature_columns].tolist()

In [119]:
DATE_STORAGE = defaultdict(dict)

In [120]:
for p in os.listdir('./'):
    if p.endswith('.csv'):
        print(p)
        get_month_dict(p)


FoodDiary_Apr_21_foods.csv
FoodDiary_Aug_21_foods.csv
FoodDiary_Feb_21_foods.csv
FoodDiary_Jan_21_foods.csv
FoodDiary_Jul_21_foods.csv
FoodDiary_Jun_21_foods.csv
FoodDiary_Mar_21_foods.csv
FoodDiary_May_21_foods.csv


In [237]:
# sorted(DATE_STORAGE.keys())

PREPROCESS

Выясним сколько дней вообще без пробелов

In [122]:
import math
import numpy as np

In [124]:
DATE_STORAGE[2][6]

{'auto_sum': [94.06, 81.32, 402.0, 2358.0],
 'raw_date': 'Saturday, January 16, 2021',
 'Breakfast': [25.54, 26.9, 91.93, 747.0],
 'Lunch': [31.62, 35.47, 146.22, 1033.0],
 'Dinner': [34.9, 18.95, 96.65, 366.0],
 'Snacks/Other': [2.0, 0.0, 67.2, 212.0]}

In [125]:
type(DATE_STORAGE[2][7]['Lunch'][0])

float

In [126]:
math.isnan(DATE_STORAGE[2][7]['Lunch'][0])

True

In [189]:
food_parts = ['Breakfast','Lunch','Dinner']
food_parts_full = ['Breakfast','Lunch','Dinner','Snacks/Other']
def isconsistent_day(day_dict):
    for f in food_parts:
        if any([math.isnan(el) for el in day_dict[f]]):
            return False
    return True

assert isconsistent_day(DATE_STORAGE[2][7]) == False

assert isconsistent_day(DATE_STORAGE[2][6]) == True


In [128]:
DATE_STORAGE[3]

{1: {'auto_sum': [90.13, 70.25, 383.93, 2088.0],
  'raw_date': 'Monday, January 18, 2021',
  'Breakfast': [23.36, 26.14, 160.23, 985.0],
  'Lunch': [32.27, 31.51, 149.6, 699.0],
  'Dinner': [34.5, 12.6, 74.1, 404.0],
  'Snacks/Other': [nan, nan, nan, nan]},
 2: {'auto_sum': [116.38, 84.8, 282.08, 2322.0],
  'raw_date': 'Tuesday, January 19, 2021',
  'Breakfast': [15.0, 9.0, 99.0, 570.0],
  'Lunch': [26.63, 9.43, 76.5, 512.0],
  'Dinner': [53.01, 58.31, 15.67, 788.0],
  'Snacks/Other': [21.74, 8.06, 90.91, 452.0]},
 3: {'auto_sum': [124.52, 95.95, 280.38, 2480.0],
  'raw_date': 'Wednesday, January 20, 2021',
  'Breakfast': [25.87, 19.62, 45.88, 460.0],
  'Lunch': [48.15, 41.53, 166.4, 1234.0],
  'Dinner': [50.5, 34.8, 68.1, 786.0],
  'Snacks/Other': [nan, nan, nan, nan]},
 4: {'auto_sum': [95.45, 90.94, 387.16, 2750.0],
  'raw_date': 'Thursday, January 21, 2021',
  'Breakfast': [26.6, 25.8, 153.7, 990.0],
  'Lunch': [34.25, 27.45, 41.4, 548.0],
  'Dinner': [34.6, 37.69, 192.06, 1212.0],

In [129]:
def is_consistent_week(week_dict):
    if len(week_dict) < 7: return False
    for day in week_dict:
        if isconsistent_day(week_dict[day]) == False:
#             print(day)
            return False
    return True
# is_consistent_week(DATE_STORAGE[2])

assert is_consistent_week(DATE_STORAGE[3]) == False

In [130]:
isconsistent_week = 0
for week in DATE_STORAGE.keys():
    if is_consistent_week(DATE_STORAGE[week]) == True:
        isconsistent_week += 1
        
isconsistent_week/len(DATE_STORAGE)

0.25

Совсем хороших неель мало. надо заполнять небитые

In [74]:
def is_week_adjustable(week_dict):
    if len(week_dict) < 4: return False
    else: return True

In [190]:
food_parts_full

['Breakfast', 'Lunch', 'Dinner', 'Snacks/Other']

In [191]:
def adjust_day(day_dict):
    list_foods = []
    for f in food_parts:
        if any([math.isnan(el) for el in day_dict[f]]):
            list_foods.append([20,20, 100, 750])
        else:
            list_foods.append(day_dict[f])
    if not any([math.isnan(el) for el in day_dict['Snacks/Other']]):
        list_foods.append(day_dict['Snacks/Other'])
    list_foods = np.array(list_foods)
    list_foods_sum = np.sum(list_foods, axis=0)
    
    adj_dict = {'adjusted_day_sum':list_foods_sum.tolist()}
    
    adj_dict = {**adj_dict, **day_dict}
    del adj_dict['auto_sum']
    return adj_dict
    
    
adjust_day(DATE_STORAGE[2][7])

{'adjusted_day_sum': [64.91, 97.00999999999999, 297.38, 2501.0],
 'raw_date': 'Sunday, January 17, 2021',
 'Breakfast': [24.91, 57.01, 97.38, 1001.0],
 'Lunch': [nan, nan, nan, nan],
 'Dinner': [nan, nan, nan, nan],
 'Snacks/Other': [nan, nan, nan, nan]}

In [192]:
def adjust_week(week_dict):
    adjusted_week = {}
    for i in range(1,8):
        if i in week_dict:
            if isconsistent_day(week_dict[i]) == True:
                adjusted_week[i] = week_dict[i]
            else:
                adjusted_week[i] = adjust_day(week_dict[i])
        else:
            adjusted_week[i] = {'full_replace_day_sum':[80, 80, 300, 2300]}
            
    return adjusted_week
        
adjust_week(DATE_STORAGE[2])

{1: {'auto_sum': [121.9, 75.46, 437.81, 2940.0],
  'raw_date': 'Monday, January 11, 2021',
  'Breakfast': [38.43, 27.5, 162.31, 1076.0],
  'Lunch': [29.26, 28.36, 67.26, 641.0],
  'Dinner': [26.35, 6.7, 90.0, 521.0],
  'Snacks/Other': [27.86, 12.9, 118.24, 702.0]},
 2: {'auto_sum': [109.11, 49.31, 412.42, 1581.0],
  'raw_date': 'Tuesday, January 12, 2021',
  'Breakfast': [33.85, 9.95, 82.61, 313.0],
  'Lunch': [41.83, 27.97, 209.13, 907.0],
  'Dinner': [33.43, 11.39, 120.68, 361.0],
  'Snacks/Other': [nan, nan, nan, nan]},
 3: {'auto_sum': [120.98, 102.14, 176.61, 1913.0],
  'raw_date': 'Wednesday, January 13, 2021',
  'Breakfast': [28.7, 33.04, 6.4, 439.0],
  'Lunch': [28.37, 7.62, 89.75, 342.0],
  'Dinner': [17.56, 44.88, 23.66, 557.0],
  'Snacks/Other': [46.35, 16.6, 56.8, 575.0]},
 4: {'auto_sum': [85.45, 60.56, 395.98, 2499.0],
  'raw_date': 'Thursday, January 14, 2021',
  'Breakfast': [26.85, 16.56, 188.83, 1026.0],
  'Lunch': [27.1, 13.25, 105.15, 662.0],
  'Dinner': [31.5, 30.7

In [193]:
adjustable_or_consistent_weeks = 0
Adjusted_date_storage = defaultdict(dict)

for week_ind, week_dict in DATE_STORAGE.items():
    if is_week_adjustable(week_dict):
        if is_consistent_week(week_dict) == True:
            Adjusted_date_storage[week_ind] = week_dict
        else:
            adjusted_week = adjust_week(week_dict)
            Adjusted_date_storage[week_ind] = adjusted_week
        
# adjustable_or_consistent_weeks/len(DATE_STORAGE) = 0.87

In [194]:
len(Adjusted_date_storage),len(DATE_STORAGE)

(29, 32)

In [195]:
sorted(Adjusted_date_storage.keys())

[2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 30,
 32,
 33,
 34]

In [196]:
collected_data_list = []
for week_ind in sorted(Adjusted_date_storage.keys()):
    week_dict = Adjusted_date_storage[week_ind]
    weekdays_consumption = []
    for week_day in range(1,8):
        for possible_daysum_key in ('auto_sum','adjusted_day_sum','full_replace_day_sum'):
            if possible_daysum_key in week_dict[week_day]:
                weekdays_consumption.append(week_dict[week_day][possible_daysum_key])
                
    mean_week_consumption = [round(el) for el in np.mean(weekdays_consumption, axis = 0).tolist()]
#     print(mean_week_consumption)
    current_week_record = [week_ind]
    current_week_record.extend(mean_week_consumption)
    
    collected_data_list.append(current_week_record)
df_collected = pd.DataFrame(data = collected_data_list, columns = ['weekind','prot', 'fat', 'carbs', 'kcal'])    

In [200]:
weekdays_consumption

[[88.77, 69.11, 311.7, 1900.0],
 [71.26, 68.64, 238.26, 2063.0],
 [76.64, 78.49, 259.57, 2026.0],
 [128.98, 75.95, 268.28, 1954.0],
 [116.65, 108.24, 273.08, 2336.0],
 [105.97, 91.33000000000001, 206.34, 2165.0],
 [67.67, 74.9, 270.59000000000003, 2194.0]]

In [202]:
df_collected.head(10)

,weekind,prot,fat,carbs,kcal
0,2,98,79,348,2271
1,3,108,91,325,2398
2,4,99,103,272,2398
3,5,106,97,262,2290
4,6,88,61,236,1770
5,7,92,70,243,2016
6,8,86,81,196,1853
7,9,88,92,223,1962
8,10,111,65,290,2005
9,11,94,74,228,1930


In [205]:
df_collected.to_csv("averaged_kkcal_consistent.csv", index = None)

In [258]:
Adjusted_date_storage[27]

{1: {'auto_sum': [82.43, 80.64, 184.04, 1798.0],
  'raw_date': 'Monday, July 5, 2021',
  'Breakfast': [23.18, 30.89, 42.46, 541.0],
  'Lunch': [39.25, 31.5, 60.48, 683.0],
  'Dinner': [20.0, 18.25, 81.1, 574.0],
  'Snacks/Other': [nan, nan, nan, nan]},
 2: {'auto_sum': [101.3, 136.48, 135.45, 2162.0],
  'raw_date': 'Tuesday, July 6, 2021',
  'Breakfast': [29.48, 35.1, 31.78, 561.0],
  'Lunch': [16.0, 14.58, 37.7, 335.0],
  'Dinner': [55.82, 86.8, 65.97, 1266.0],
  'Snacks/Other': [nan, nan, nan, nan]},
 3: {'adjusted_day_sum': [63.97, 76.41, 211.85, 1878.0],
  'raw_date': 'Wednesday, July 7, 2021',
  'Breakfast': [28.26, 36.92, 36.9, 592.0],
  'Lunch': [15.71, 19.49, 74.95, 536.0],
  'Dinner': [nan, nan, nan, nan],
  'Snacks/Other': [nan, nan, nan, nan]},
 4: {'full_replace_day_sum': [80, 80, 300, 2300]},
 5: {'auto_sum': [52.32, 96.78, 137.97, 1640.0],
  'raw_date': 'Friday, July 9, 2021',
  'Breakfast': [23.98, 31.3, 1.88, 385.0],
  'Lunch': [15.16, 46.08, 86.77, 825.0],
  'Dinner': 

Merge with weight

In [232]:
from datetime import datetime

In [254]:
timestamp = 1598426894
dt_object = datetime.fromtimestamp(timestamp)

print("dt_object =", dt_object)
print("type(dt_object) =", type(dt_object))

dt_object = 2020-08-26 10:28:14
type(dt_object) = <class 'datetime.datetime'>


In [238]:
dt_object.isocalendar()

(2021, 12, 2)

In [235]:
df_weight = pd.read_csv("BODY_1630239177349.csv")

In [236]:
df_weight.head()

,timestamp,weight,height,bmi,fatRate,bodyWaterRate,boneMass,metabolism,muscleRate,visceralFat,impedance
0,1616475110,92.25,176.0,29.781120,0.0,0.0,0.00,0.0,0.00,0.0,0
1,1616392684,91.55,176.0,29.555140,29.9,49.9,3.26,1873.0,60.83,12.0,472
2,1616132239,92.65,176.0,29.910254,0.0,0.0,0.00,0.0,0.00,0.0,0
3,1616045319,91.90,176.0,29.668130,0.0,0.0,0.00,0.0,0.00,0.0,0
4,1616045287,91.95,176.0,29.684270,0.0,0.0,0.00,0.0,0.00,0.0,0
